In [134]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


In [135]:
# Load the image
name="banana3"

path="../data/section-2-grabcut/images/"+name+".jpg"
img = cv2.imread(path)
#img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
filename = (path.split("/")[-1]).split(".")[0]

In [136]:

def draw_rectangle(image):
    
    cv2.namedWindow("Image")
    cv2.imshow("Image", image)

    
    x1, y1, x2, y2 = -1, -1, -1, -1
    drawing = False

    
    def mouse_callback(event, x, y, flags, param):
        nonlocal x1, y1, x2, y2, drawing

        if event == cv2.EVENT_LBUTTONDOWN:
            x1, y1 = x, y
            x2, y2 = x, y
            drawing = True

        elif event == cv2.EVENT_MOUSEMOVE:
            if drawing:
                x2, y2 = x, y

        elif event == cv2.EVENT_LBUTTONUP:
            x2, y2 = x, y
            drawing = False

    cv2.setMouseCallback("Image", mouse_callback)

    while True:
        img = image.copy()

        if x2 >= 0 and y2 >= 0:
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

        cv2.imshow("Image", img)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('c'):
            x1, y1, x2, y2 = -1, -1, -1, -1

    cv2.destroyAllWindows()
    print(x1,y1,x2,y2)
    return (x1, y1, x2, y2)


In [137]:
def calculate_accuracy(mine, gt):
    out_mask = np.uint8(mine != 0)
    gt_mask = np.uint8(gt != 0)

    accuracy = np.mean(out_mask == gt_mask)

    intersection = np.sum(np.logical_and(out_mask, gt_mask))
    union = np.sum(np.logical_or(out_mask, gt_mask))
    jaccard = intersection / union

    dice = 2 * intersection / (np.sum(out_mask) + np.sum(gt_mask))

    print("Accuracy: ", accuracy, "\nJaccard: ", jaccard, "\nDice: ", dice)


In [138]:
x1, y1, x2, y2 = draw_rectangle(img)
rect = (min(x1, x2), min(y1, y2), abs(x2-x1), abs(y2-y1))
# rect = (68, 153, 338, 500) # memorial
# rect=(143,27,325,452) #tennis
# rect=(117,109,356,519) #llama
# rect-(118,48, 353, 537) #bush
# rect = (155, 128, 297, 517) #person6

mask = np.zeros(img.shape[:2], np.uint8)
bgdModel = np.zeros((1, 65), np.float64)
fgdModel = np.zeros((1, 65), np.float64)

cv2.grabCut(img, mask, rect, bgdModel, fgdModel, 10, cv2.GC_INIT_WITH_RECT)
mask2 = np.where((mask == 2) | (mask == 0), 0, 255).astype('uint8')

41 65 594 414


In [139]:
outpath="../results/section-2-grabcut/"+filename+".png"
gtpath="../data/section-2-grabcut/ground_truth/"+filename+".bmp"

cv2.imwrite(outpath, mask2)
calculate_accuracy(cv2.imread(outpath), cv2.imread(gtpath))


Accuracy:  0.9825130208333334 
Jaccard:  0.9294411243186446 
Dice:  0.9634304074936351
